In [6]:
from datetime import datetime
import os
from scipy.io import loadmat
import glob
import numpy as np



In [7]:
def determine_maze_size(mat_file_name):
    # Assuming mat_file_name format includes a date and time like "Hermann_20210217T1510"
    date_str = mat_file_name.split('_')[1]  # Extracts "20210217T1510"
    date_str = date_str.split('T')[0]  # Now extracts only the date part "20210217"
    
    # Convert the string to a datetime object for accurate date comparison
    date_obj = datetime.strptime(date_str, "%Y%m%d")  # Converts string to datetime object
    
    # Compare datetime objects
    if date_obj <= datetime(2021, 3, 18):
        return 6  # 6x6 maze for dates up to and including March 18, 2021
    else:
        return 7  # 7x7 maze for dates after March 18, 2021

In [8]:
def extract_binary_strings(mazewall):
    wall_state = mazewall['hidden'][0, 0]  # Accessing the wall state data
    A = []
    for row in wall_state.flatten():  # Flatten the structure if it's multidimensional
        value = row.item()  # Extract the integer value directly
        A.append(value)

    binary_strings = [''.join(str(A[j]) for j in range(i, min(i + 4, len(A)))) for i in range(0, len(A), 4)]
    return binary_strings

In [9]:
def coordinate_to_binary_string_index(coord, maze_size):
    # Calculate the linear index for the corresponding binary string.
    # This formula accounts for the described column-wise bottom-to-top, left-to-right traversal.
    index = coord[1] + coord[0] * maze_size
    return index


def determine_direction(curr, next_pos):
    # Determine the direction of movement from curr to next_pos.
    delta_x, delta_y = next_pos[0] - curr[0], next_pos[1] - curr[1]
    if delta_x == 1:
        return "right"
    elif delta_x == -1:
        return "left"
    elif delta_y == 1:
        return "up"
    elif delta_y == -1:
        return "down"
    return "stay"

In [10]:
def process_single_file(mat_file_path, output_directory):
    mat_data = loadmat(mat_file_path)  # Load the data from the .mat file
    mat_file_name = os.path.splitext(os.path.basename(mat_file_path))[0]
    maze_size= determine_maze_size(mat_file_name)
    # define the specific directory for deicison points txt files
    decision_points_directory = os.path.join(output_directory, 'MarchMarchtrajectory_txt_file')
    os.makedirs(decision_points_directory, exist_ok=True)

    for trial_index in range(mat_data['data'].shape[0]):  # Process each trial
        cell = mat_data['data'][trial_index][0]
        trajectory_cell_path=cell['trajectory_cell_path']
        coordinate_pairs_array = trajectory_cell_path[0][0]
        # # Now, iterate through the coordinate pairs to create the list of tuples
        # trajectory = [(int(x), int(y)) for [x, y] in coordinate_pairs_array]
        if coordinate_pairs_array.size==0:
            trajectory=[]
        else:
            trajectory = [(int(x), int(y)) for [x, y] in coordinate_pairs_array]
        
            
        maze_data=cell['maze'][0,0]
        mazewall = cell['mazewall'][0,0]
        binary_strings = extract_binary_strings(mazewall)    
        # Define the output path for this trial's decision points text file
        txt_output_path = os.path.join(decision_points_directory, f"{mat_file_name}_maze_{maze_size}x{maze_size}_trial_{trial_index}_decision_points.txt")
        
        # incorrect 
        with open(txt_output_path, 'w') as file:
            file.write("prev_x, prev_y,current_x,current_y,next_x,next_y,direction\n")  
            if len(trajectory)<2:
                file.write("empty\n")
            else:
                for i in range(1,len(trajectory) - 1):
                    prev = trajectory[i - 1]
                    curr = trajectory[i]
                    next_pos = trajectory[i + 1]
        # Determine direction (this function should be defined as shown previously)
                    direction = determine_direction(curr, next_pos)
                    index = coordinate_to_binary_string_index(curr,maze_size)
                    if index>=0 and index < len(binary_strings):
                        corresponding_binary_string = binary_strings[index]
                        open_paths_count = corresponding_binary_string.count('1')
                        is_decision_point = open_paths_count > 1
                        if is_decision_point:
                # Write only decision points to the file
                            file.write(f"{prev[0]},{prev[1]},{curr[0]},{curr[1]},{next_pos[0]},{next_pos[1]},{direction}\n")



        print(f"Generated decision points text file: {txt_output_path}")

        


        
       
# Main script
directory_path = '/Users/cindy/Desktop/monkeymaze/monkey_data/preprocessed/Hermann/Hermann_detour_data/'
output_directory = '/Users/cindy/Desktop/monkeymaze/'
decision_points_directory = os.path.join(output_directory, 'March20trajectory_txt_file')
os.makedirs(decision_points_directory, exist_ok=True)
txt_directory = os.path.join(output_directory, 'March20trajectory_txt_files')
mat_files = glob.glob(os.path.join(directory_path, '*.mat'))

for mat_file in mat_files:
    process_single_file(mat_file, output_directory)  # This function now handles both txt and png outputs


Generated decision points text file: /Users/cindy/Desktop/monkeymaze/March20trajectory_txt_file/Hermann_20210301T151026_detour_nodes_maze_6x6_trial_0_decision_points.txt
Generated decision points text file: /Users/cindy/Desktop/monkeymaze/March20trajectory_txt_file/Hermann_20210301T151026_detour_nodes_maze_6x6_trial_1_decision_points.txt
Generated decision points text file: /Users/cindy/Desktop/monkeymaze/March20trajectory_txt_file/Hermann_20210301T151026_detour_nodes_maze_6x6_trial_2_decision_points.txt
Generated decision points text file: /Users/cindy/Desktop/monkeymaze/March20trajectory_txt_file/Hermann_20210301T151026_detour_nodes_maze_6x6_trial_3_decision_points.txt
Generated decision points text file: /Users/cindy/Desktop/monkeymaze/March20trajectory_txt_file/Hermann_20210301T151026_detour_nodes_maze_6x6_trial_4_decision_points.txt
Generated decision points text file: /Users/cindy/Desktop/monkeymaze/March20trajectory_txt_file/Hermann_20210301T151026_detour_nodes_maze_6x6_trial_5_

KeyboardInterrupt: 